### **URL 더 똑똑하게 사용하기**

이번에는 템플릿에서 사용한 URL 하드 코딩을 없애는 방법에 대해 알아보자.
URL 하드 코딩이란 무엇일까? question_list.html 템플릿에 사용된 href 값을 보자.

```
<li><a href="/pybo/{{ question.id }}//">{{ question-subject }}</a></li>
```

"/pybo/{{ question.id }}"는 질문 상세를 위한 URL 규칙이다. 하지만 이런 URL 규칙은 프로그램을 수정하면서 '/pybo/question/2/' 또는
'/pybo/2/question/'으로 수정될 가능성도 있다. 이런 식으로 URL 규칙이 자주 변경된다면 모든 href 값들을 일일이 찾아 수정해야 한다.
URL 하드 코딩의 한계인 셈이다.
이런 문제를 해결하려면 해당 URL에 대한 실제 주소가 아닌 주소가 매핑된 URL 별칭을 사용해야 한다.

#### **URL 별칭으로 URL 하드 코딩 문제 해결하기**

* pybo/urls.py 수정하여 URL 별칭 사용하기

템플릿의 href에 실제 주소가 아닌 URL 별칭을 사용하려면 우선 pybo/urls.py 파일을 수정해야 한다.
path 함수에 있는 URL 매핑에 name 속성을 부여한다.

```
from django.urls import path

from . import views


urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detal'),
]
```

이렇게 수정하면 실제 주소 /pybo/는 index라는 URL 별칭이, /pybo/2/는 detail이라는 URL 별칭이 생긴다.

* pybo/question_list.html 템플릿에서 URL 별칭 사용하기

1단계에서 만든 별칭을 템플릿에서 사용하기 위해 /pybo/{{ question.id }}를 {% url 'detail' question.id %}로 변경한다.
(question.id는 URL 매핑에 정의된 <int:question_id>를 의미한다)

```
(...생략...)
    {% for question in question_list %}
        <li><a href="{% url 'detail' question.id %}">{{ question,subject }}</a></li>
    {% endfor %}
(...생략...)
```

#### **URL 네임스페이스 알아보기**

여기서 한 가지 더 생각할 문제가 있다.
현재 프로젝트에서는 pybo 앱 하나만 사용하지만 이후 pybo 앱 이외의 다른 앱이 프로젝트에 추가될 수도 있다.
이때 서로 다른 앱에서 같은 URL 별칭을 사용하면 중복 문제가 발생한다.

이 문제를 해결하려면 pybo/urls.py 파일에 네임스페이스(namespace)라는 개념을 도입해야 한다.
네임스페이스는 쉽게 말해 각각의 앱이 관리하는 독립된 이름 공간을 의미한다.

* pybo/urls.py에 네임스페이스 추가하기

pybo/urls.py 파일에 네임스페이스를 추가하려면 간단히 app_name 변수에 네임스페이스 이름을 저장하면 된다.

```
from django.urls import path
from . import views

app_name = 'pybo'

urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detal'),
]
```

네임스페이스 이름으로 'pybo'를 저장했다.

* 네임스페이스 테스트하기 - 오류 발생!

/pybo/에 접속해보자.
하지만 오류가 발생한다.

* pybo/question_list.html 수정하기

오류가 발생한 이유는 템플릿에서 아직 네임스페이스를 사용하고 있지 않기 때문이다.
{% url 'detail' question.id %}을 {% url 'pybo:detail' question.id %}로 바꾸자.

```
{% for question in question_list %}
    <li><a href="/pybo/{{ question.id }}/">{{ question.subject }}</a></li>
    <li><a href="{% url 'pybo:detail' question.id %}">{{ question.subject }}</a></li>
{% endfor %}
```

detial에 pybo라는 네임스페이스를 붙여준 것이다.

### **답변 등록 기능 만들기**

앞에서 질문 등록, 조회 기능을 만들었다.
이번에는 답변 등록과 답변을 보여주는 기능을 만들어 보자.

#### 답변 저장하고 표시하기

* 질문 상세 템플릿에 답변 등록 버튼 만들기

질문 상세 템플릿 pybo/question_detail.html 파일을 수정한다.
form 엘리먼트 안에 textarea 엘리먼트와 input 엘리먼트를 포함시켜 답변 내용, 답변 등록 버튼을 추가한다.

```
<h1>{{ question.subject }}</h1>

<div>
    {{ question.content }}
</div>

<form action="{% url 'pybo:answer_create' question.id} %" method="post">
    {% csrf_token %}
    <textarea name="content" id="content" rows="15"></textarea>
    <input type="submit" value="답변 등록">
</form>
```

<답변 등록> 버튼을 누를 때 호출되는 URL은 action 속성에 있는 {% url 'pybo:answer_create' question.id %}이다.
그리고 form 엘리먼트 바로 아래에 있는 {% csrf_token %}이 눈에 띈다.
이 코드는 보안 관련 항목이니 잠시 뒤에 설명하겠다.
{% csrf_token %}는 form 엘리먼트를 통해 전송된 데이터(답변)가 실제로 웹 브라우저에서 작성된 데이터인지 판단한다.
그러므로 <form ...> 태그 바로 밑에 {% csrf_token %}을 항상 입력해야 한다. 
해킹처럼 올바르지 않은 방법으로 데이터가 전송되면 서버에서 발행한 csrf_token 값과 해커가 보낸 csrf_token 값이 일치하지 않으므로 오류를 발생시켜
보안을 유지할 수 있다.

* 답변 등록을 위한 URL 매핑 등록하기

pybo/urls.py 파일에 답변 등록을 위한 URL 매핑을 등록한다.

```
urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detal'),
    path('answer/create/<int:question_id>/', views.answer_create, name='answer_create'),
]
```

이 코드는 사용자가 상세 화면에서 <질문 답변> 버튼을 눌렀을 때 작동할 form 엘리먼트의 pybo/answer/create/2/에 대한 
URL 매핑을 추가한 것이다.

* answer_create 함수 추가하기

form 엘리먼트에 입력된 값을 받아 데이터베이스에 저장할 수 있도록 answer_create 함수를 pybo/views.py 파일에 추가한다.

```
from django.shortcuts import (
    render,
    get_object_or_404,
    redirect,
)
from .models import Question
from django.utils import timezone


def answer_create(request, question_id):
    """
    pybo 답변 등록
    """
    
    question = get_object_or_404(Question, pk=question_id)
    question.answer_set.create(
        content = request.POST.get('content'),
        create_date = timezone.now(),
    )
```

answer_create 함수의 question_id 매개변수에는 URL 매핑 정보값이 넘어온다.
예컨대 /pybo/answer/create/2가 요청되면 question_id에는 2가 넘어온다.
request 매개변수에는 pybo/question_detial.html에서 textarea에 입력된 데이터가 파이썬 객체에 담겨 넘어온다.
이 값을 추출하기 위한 코드가 바로 request.POST.get('content')이다.

그리고 Question 모델을 통해 Answer 모델 데이터를 생성하기 위해 question.answer_set.create를 사용했다.

```
* Answer 모델을 통해 데이터를 저장할 수도 있다.

위에서는 Answer 모델 데이터 저장을 위해 Question 모델을 사용했지만,
Answer 모델을 직접 사용해도 저장할 수 있다.

    # answer = Answer(question=question, content=request.POST.get('content'), create_date=timezone.now())
    # answer.save()
```

* 답변 등록 후 상세 화면으로 이동하게 만들기

답변을 생성한 후 상세 화면을 호출하려면 redirect 함수를 사용하여 코드를 작성하면 된다.
redirect 함수는 함수에 전달된 값을 참고하여 페이지 이동을 수행한다.
redirect 함수의 첫 번째 인수에는 이동할 페이지의 별칭을, 두 번째 인수에는 해당 URL에 전달해야 하는 값을 입력한다.

```
from django.shortcuts import (
    render,
    get_object_or_404,
    redirect,
)
from .models import Question
from django.utils import timezone


def answer_create(request, question_id):
    """
    pybo 답변 등록
    """
    
    question = get_object_or_404(Question, pk=question_id)
    # answer = Answer(question=question, content=request.POST.get('content'), create_date=timezone.now())
    # answer.save()
    question.answer_set.create(
        content = request.POST.get('content'),
        create_date = timezone.now(),
    )
    return redirect(
        'pybo:detail',
        question_id=question_id
    )
```

* 등록된 답변 표시하기

질문 상세 화면에 답변을 표시하려면 pybo/question_detail.html 파일을 수정해야 한다.

```
<h1>{{ question.subject }}</h1>

<div>
    {{ question.content }}
</div>

<h5>{{ question.answer_set.count }}개의 답변이 있습니다.</h5>
<div>
    <ul>
        {% for answer in question.answer_set.all %}
        <li>{{ answer.content }}</li>
        {% endfor %}
    </ul>
</div>

<form action="{% url 'pybo:answer_create' question.id} %" method="post">
    {% csrf_token %}
    <textarea name="content" id="content" rows="15"></textarea>
    <input type="submit" value="답변 등록">
</form>
```

question.answer_set.count는 답변 개수를 의미한다.
질문 내용과 답변 입력 창 사이에 답변 표시 영역을 추가했다.
코드를 위처럼 수정한 후에 질문 상세 페이지에 접속하면 답변 저장, 답변 조회 기능을 완성할 수 있다.

### **화면 예쁘게 꾸미기**

지금까지 질문과 답변을 등록하고 조회하는 기능을 만들었다.
그런데 그럴싸한 화면이 아니라서 아쉽다.
여기서는 스타일시트를 이용해 웹 페이지에 디자인을 적용하는 방법을 알아보도록 하겠다.

#### **웹 페이지에 스타일시트 적용하기**

웹 페이지에 디자인을 적용하려면 스타일스트(css)를 사용해야 한다.

* 설정 파일에 스태틱 디렉터리 위치 추가하기

config/settings.py 파일을 열어 STATICFILES_DIRS에 스태틱 디렉터리 경로를 추가한다.
BASE_DIR / 'static'은 C:/projects/mysite/static을 의미한다.

```
STATIC_URL = "static/"
STATICFILES_DIRS = [
    BASE_DIR / 'static',
]
```

* 스태틱 디렉터리를 만들고 스타일시트 작성하기

프로젝트 루트 디렉터리에 static이라는 이름의 디렉터리를 생성한다.
루트 디렉터리는 django_study/mysite를 의미한다.

```
mkdir static
```

pybo 앱 디렉터리 바로 아래에 static 디렉터리를 만들어도 장고에서 스태틱 디렉터리로 인식된다.
즉, 다음과 같이 static 디렉터리를 만들어도 된다.

```
mysite/pybo mkdir static
```

하지만 이 방법 역시 템플릿 디렉터리를 구성할 때 설명했듯 프로젝트 관리를 불편하게 만든다.
스태틱 디렉터리도 템플릿 디렉터리처럼 한 곳으로 모아서 관리할 것이다.

static 디렉터리를 만들었으면 그곳에 style.css 파일을 만들어 다음 코드를 작성한다.
여기서는 답변을 등록할 때 사용하는 textarea를 100%으로 넓히고, <답변 등록> 버튼 위에 margin을 10px 추가했다.

```
textarea {
    width: 100%;
}

input[type=submit] {
    margin-top: 10px;
}
```

* 질문 상세 템플릿에 스타일 적용하기

pybo/question_detail.html 파일에 style.css 파일을 적용해 본다.
스태틱 파일을 사용하기 위해 템플릿 파일 맨 위에 {% load static %} 태그를 삽입하고, link 엘리먼트 href 속성에 
{% static 'style.css' %}를 적는다.

```
{% load static %}
<link rel="stylesheet" type="text/css" href="{% static 'style.css' %}">
<h1>{{ question./subject }}</h1>
```

추가한 코드는 static 디렉터리의 style.css 파일을 연결한다는 의미이다.

### **부트스트랩으로 더 쉽게 화면 꾸미기**

웹 디자이너 없이 웹 프로그램을 만들다 보면 화면 디자인 작업을 하는 데 얼마나 많은 시간과 고민이 필요한지 알 수 있을 것이다.
이번에 소개하는 부트스트랩(Bootstrap)은 개발자 혼자서도 화면을 괜찮은 수준으로 만들 수 있게 도와주는 도구이다.
부트스트랩은 트위터를 개발하면서 만들어졌고 지속적으로 관리되고 있는 오픈소스 프로젝트다.

#### **파이보에 부트스트랩 적용하기**

웹 브라아주에서 부트스트랩 4.5.3 버전을 다운로드하도록 한다.
내려받은 파일의 압축을 해체하면 많은 파일들이 들어 있다.
그중에서 bootstrap.min.css 파일만 복사해서 mysite/static 디렉터리에 저장한다.

* 질문 목록 템플릿에 부트스트랩 적용하기

pybo/question_list.html 파일에 부트스트랩을 적용하기 위해 {% load static %} 태그와 link 엘리먼트를 추가한다.

```
{% load static %}
<link rel="stylesheet" type="text/css" href="{% static 'bootstrap.min.css' %}">
{% if question_list %}
```

이어서 부트스트랩을 이용하여 템플릿 화면을 다음과 같이 재구성한다.
여기서 사용된 container, my-3, thead-dark 등이 바로 부트스트랩이 제공하는 클래스다.

```
{% if question_list %}   
    <ul>
    {% for question in question_list %}
        <li><a href="/pybo/{{ question.id }}/">{{ question.subject }}</a></li>
        <li><a href="{% url 'pybo:detail' question.id %}">{{ question.subject }}</a></li>
    {% endfor %}    
    </ul>
{% else %}  
    <p>질문이 없습니다.</p>
{% endif %}

{% load static %}
<link rel="stylesheet" type="text/css" href="{% static 'bootstrap.min.css' %}">
<div class="container my-3">
    <table class="table">
        <thead>
            <tr class="thead-dark">
                <th>번호</th>
                <th>제목</th>
                <th>작성일시</th>
            </tr>
        </thead>
        <tbody>
        {% if question_list %}
        {% for question in question_list %}
        <tr>
            <td>{{ forloop.counter }}</td>
            <td>
                <a href="{% url 'pybo:detail' question.id %}">
                    {{ question.subject }}
                </a>
            </td>
            <td>{{ question.create_date }}</td>
        </tr>
        {% endfor %}
        {% else %}
        <tr>
            <td colspan="3">질문이 없습니다.</td>
        </tr>
        {% endfor %}
    </tbody>
    </table>
</div>
```

기존에는 질문 목록을 ul 엘리먼트로 간단히 표시했지만, 여기서는 table 엘리먼트로 바꾸고 
질문의 일련번호와 작성일시 항목도 추가했다. 질문의 일련번호는 {{ forloop.conuter }}를 이용하여 표시했다.
{{ forloop.counter }}는 {% for ... %}에서 반복 시 자동으로 매겨지는 순서값을 의미한다.
웹 브라우저에서 /pybo에 접속하면 부트스트랩이 적용된 화면을 볼 수 있다.

* 질문 상세 템플릿에 부트스트랩 사용하기

질문 상세 템플릿도 다음과 같이 부트스트랩을 적용한다.

```
<h1>{{ question.subject }}</h1>

<div>
    {{ question.content }}
</div>

{% load static %}
<link rel="stylesheet" type="text/css" href="{% static 'bootstrap.min.css' %}">
<div class="container my-3">
    <h2 class="border-bottom py-2">{{ question.subject }}</h2>
    <div class="card my-3">
        <div class="card-body">
            <div class="card-text" style="white-space: pre-line;">
                {{ question.content }}
            </div>
            <div class="d-flex justify-content-end">
                <div class="badge badge-light p-2">
                    {{ question.create_date }}
                </div>
            </div>
        </div>
    </div>
    <h5 class="border-bottom my-3 py-2">
        {{ question.answer_set.count }}개의 답변이 있습니다.
    </h5>
    {% for answer in question.answer_set.all %}
    <div class="card my-3">
        <div class="card-text" style="white-space: pre-line;">
            {{ answer.content }}
        </div>
        <div class="d-flex justify-content-end">
            <div class="badge badge-light p-2">
                {{ answer.create_date }}
            </div>
        </div>
    </div>
</div>
{% endfor %}
<form action="{% url 'pybo:answer_create' question.id %}"
    method="post" class="my-3">
    {% csrf_token %}
    <div class="form-group">
        <textarea name="content" id="content" class="form-control" rows="10"></textarea>
    </div>
    <input type="submit" value="답변 등록" class="btn btn-primary">
</form>
```

질문, 답변은 하나의 뭉치에 해당되므로 부트스트랩의 card 컴포넌트를 사용했고, 질문과 답변 내용은 style 속성으로 
white-space: pre-line을 적용하여 텍스트의 줄바꿈을 정상적으로 보이게 만들었다.
부트스트랩 클래스 my-3은 상하 마진값 3을 의미한다.
py-2는 상하 패딩값, p-2는 상하좌우 패딩값 2를 의미한다.
d-flex justify-content-end는 컴포넌트 오른쪽 정렬을 의미한다.

### **표준 HTML과 템플릿 상속 사용해 보기**

혹시 눈치챘는지 모르겠지만, 지금까지 작성한 질문 목록과 질문 상세 템플릿 파일은 표준 HTML 구조가 아니다.
어떤 운영체제나 웹 브라우저를 사용하더라도 웹 페이지가 동일하게 보이고 정상적으로 작동하게 하려면 반드시 웹 표준을 지키는 HTML 문서를 작성해야 한다.

**표준 HTML 구조는 어떻게 생겼을까?**

표준 HTML 문서의 구조는 다음과 같이 html, head, body 엘리먼트가 있어야 하며, CSS 파일은 head 엘리먼트 안에 있어야 한다.
또한 head 엘리먼트 안에는 meat, title 엘리먼트 등이 포함되어야 한다.

#### **템플릿을 표준 HTML 구조로 바꾸기**

앞에서 작성한 템플릿 파일을 표준 HTML 구조로 수정해 본다.
그런데 모든 템플릿 파일을 표준 HTML 구조로 변경하면 body 엘리먼트 바깥 부분은 모두 같은 내용으로 중복된다.
그리고 CSS 파일 이름이 변경되거나 새로운 CSS 파일이 추가되면 head 엘리먼트의 내용을 수정하려고 템플릿 파일을 일일이 찾아다녀야 하는 불편함도 있다.

장고는 이런 불편함을 해소하기 위한 템플릿 상속(extends) 기능을 제공한다.
여기서는 단순히 템플릿을 HTML 구조로 바꾸는 것이 아니라 템플릿 상속 기능까지 사용할 것이다.
그러면 파일을 하나씩 수정해본다.

* 템플릿 파일의 기본 틀 작성하기

우선 템플릿 파일의 기본 틀인 base.html 템플릿을 작성해본다.
모든 템플릿에서 공통으로 입력할 내용을 여기에 포함한다고 생각하면 된다.

```
{% load static %}
<!doctpye html>
<html lang="ko">
<head>
    <!--Requried meta tags-->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <!--Bootstrap CSS-->
    <link rel="stylesheet" type="text/css" href="{% static 'bootstrap.min.css' %}">
    <!--pybo CSS-->
    <link rel="stylesheet" type="text/css" href="{% static 'style.css' %}">
    <title>Hello, pybo!</title>
</head>
<body>
<!--기본 템플릿 안에 삽입될 내용 Start-->
{% block content %}
{% endblock %}
<!--기본 템플릿 안에 삽입될 내용 End-->
</body>
</html>
```

body 엘리먼트에 {% block content %}와 {% endblock %} 템플릿 태그가 있다.
바로 이 부분이 이후 base.html 템플릿 파일을 상속한 파일에서 구현해야 하는 영역이 된다.
이제 question_list.html 템플릿을 다음과 같이 변경한다.

* 질문 목록 템플릿 수정하기

질문 목록을 나타내는 question_list.html 파일을 다음과 같이 수정한다.

```

```